In [ ]:
%%capture
!pip install koreanize-matplotlib
import koreanize_matplotlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
chicago = pd.read_csv('/content/drive/MyDrive/chicago_crime_final.csv')

In [ ]:
c_name = pd.read_csv('/content/drive/MyDrive/community_areas_final.csv')

In [ ]:
def is_violent_crime(row):
    primary = row['Primary Type']
    description = str(row['Description']).upper()

    if primary in ['HOMICIDE', 'CRIMINAL SEXUAL ASSAULT', 'ROBBERY', 'HUMAN TRAFFICKING']:
        return True
    elif primary in ['ASSAULT', 'BATTERY', 'ARSON'] and 'AGGRAVATED' in description:
        return True
    else:
        return False

chicago['IsViolent'] = chicago.apply(is_violent_crime, axis=1)

In [ ]:
# 위험범죄만 필터링
severe_crime_df = chicago[chicago['IsViolent']]

In [ ]:
# 1. 위험범죄 건수 by Community Area
severe_by_area = severe_crime_df.groupby('Community Area').size().reset_index(name='Severe Count')
severe_by_area['Community Area'] = severe_by_area['Community Area'].astype(int)

# 2. 인구와 병합 + 비율
severe_merge = pd.merge(severe_by_area, c_name, how='left', on='Community Area')
severe_merge['Severe Rate'] = severe_merge['Severe Count'] / severe_merge['population']

In [ ]:
# 3. KMeans
from sklearn.cluster import KMeans
features = severe_merge[['Severe Rate']]
kmeans = KMeans(n_clusters=3, random_state=42)
severe_merge['Cluster'] = kmeans.fit_predict(features)

In [ ]:
# 4. 시각화
import plotly.express as px
fig = px.scatter(
    severe_merge,
    x='Severe Count',
    y='Severe Rate',
    color='Cluster',
    hover_name='name',
    title='흉악범죄 기준 위험지역 클러스터링'
)
fig.show()